# Open Trust Fabric (OTF)
# Digital Platform Use Case
# 03- House Prices Analysis
June 2021

# Description

This notebook makes a comparison of AirBnB digital platform contracts and the house prices registered across different European cities.

House prices are collected from Idealista, a real estate website focused in Spain and Italy. House prices were downloaded from this website: https://www.idealista.it/data/

The purpose of this analysis is to identify whether there is a potential correlation across the number of contracts carried out in the AirBnB digital platform and the house prices in idealista.

House prices are captured as euros per square meter. The analysis will be carried out for two cities: Madrid and Rome.

In [1]:
import pandas as pd
import numpy as np
import os
import gzip
from functools import reduce

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [2]:
data_path = '../../../ecme/UC1: Digital Platform/Data/'

In [3]:
with gzip.open(data_path + 'POLICY_NETWORK.csv.gzip') as file_in:
    df = pd.read_csv(file_in, low_memory=False)

In [4]:
#ASSET_PRICE
def fix_price (value):
    try:
        value = value.replace("$", '').replace(",", '')
        value = float(value)
        return value
    except: return np.nan  

df['ASSET_PRICE'] = df['ASSET_PRICE'].apply(lambda x : fix_price(x))

In [5]:
df_price_bnb_rome = df.loc[df.ASSET_LOCATION == 'rome'].groupby(['YEAR', 'MONTH', "ASSET_ID"])[["ASSET_PRICE"]].mean().reset_index()
df_price_bnb_rome = df_price_bnb_rome.groupby(['YEAR', 'MONTH']).agg({"ASSET_PRICE" : "median"})
df_price_bnb_rome = df_price_bnb_rome.reset_index()

In [6]:
df_price_bnb_madrid = df.loc[df.ASSET_LOCATION == 'madrid'].groupby(['YEAR', 'MONTH', "ASSET_ID"])[["ASSET_PRICE"]].mean().reset_index()
df_price_bnb_madrid = df_price_bnb_madrid.groupby(['YEAR', 'MONTH']).agg({"ASSET_PRICE" : "median"})
df_price_bnb_madrid = df_price_bnb_madrid.reset_index()

In [7]:
# IDEALISTA DATA

In [8]:
# ROME

In [9]:
df_rome = pd.read_csv('rome_ts.csv', delimiter=";")
df_rome.columns = [c.upper() for c in df_rome.columns]

In [10]:
df_rome.head()

,MONTH,YEAR,EURO_MQ
0,Gen,2012,4110
1,Feb,2012,4098
2,Mar,2012,4113
3,Apr,2012,4101
4,Mag,2012,4057


House prices data 

In [11]:
df_rome.shape

(107, 3)

In [12]:
month_mapper = {
    'Gen' : 1, 'Feb' : 2, 'Mar' : 3, 'Apr' : 4,  'Mag' : 5,  'Giu' : 6,
    'Lug' : 7, 'Ago' : 8, 'Set' : 9, 'Ott' : 10, 'Nov' : 11, 'Dic' : 12
        }

In [13]:
df_rome['N_MONTH'] = df_rome.MONTH.map(month_mapper)

In [14]:
df_rome.head(5)

,MONTH,YEAR,EURO_MQ,N_MONTH
0,Gen,2012,4110,1
1,Feb,2012,4098,2
2,Mar,2012,4113,3
3,Apr,2012,4101,4
4,Mag,2012,4057,5


In [15]:
df_rome.dtypes

MONTH      object
YEAR        int64
EURO_MQ     int64
N_MONTH     int64
dtype: object

In [17]:
df_rome.head()

,MONTH,YEAR,EURO_MQ,N_MONTH
0,Gen,2012,4110,1
1,Feb,2012,4098,2
2,Mar,2012,4113,3
3,Apr,2012,4101,4
4,Mag,2012,4057,5


In [ ]:
# MADRID

In [18]:
df_madrid = pd.read_csv('Madrid_ts.csv', delimiter=";")

In [19]:
month_mapper = {
    'Ene' : 1, 'Feb' : 2, 'Mar' : 3, 'Abr' : 4,  'May' : 5,  'Jun' : 6,
    'Jul' : 7, 'Ago' : 8, 'Sep' : 9, 'Oct' : 10, 'Nov' : 11, 'Dic' : 12
        }

In [20]:
df_madrid['N_MONTH'] = df_madrid.MONTH.map(month_mapper)

In [21]:
df_madrid.EURO_MQ = df_madrid.EURO_MQ.str.strip()

In [22]:
df_madrid.EURO_MQ.replace('null', np.nan, inplace=True)

In [23]:
df_madrid.EURO_MQ = df_madrid.EURO_MQ.astype(float)

In [24]:
df_madrid.head(12)

,MONTH,YEAR,EURO_MQ,N_MONTH
0,Ene,2006,NaN,1
1,Feb,2006,NaN,2
2,Mar,2006,3586.0,3
3,Abr,2006,3431.0,4
4,May,2006,3550.0,5
5,Jun,2006,3473.0,6
6,Jul,2006,3488.0,7
7,Ago,2006,3477.0,8
8,Sep,2006,3458.0,9
9,Oct,2006,3359.0,10


In [25]:
df_merged_rome = df_rome[[c for c in list(df_rome) if c !='MONTH']].merge(df_price_bnb_rome, left_on = ['YEAR', 'N_MONTH'], 
                          right_on = ['YEAR', 'MONTH'], how='inner')

In [26]:
df_merged_rome.shape

(10, 5)

In [27]:
df_merged_rome

,YEAR,EURO_MQ,N_MONTH,MONTH,ASSET_PRICE
0,2017,3006,5,5,79.0
1,2018,2971,4,4,79.0
2,2018,2948,5,5,79.0
3,2018,2947,6,6,79.0
4,2018,2973,7,7,80.0
5,2018,2950,8,8,80.0
6,2018,2923,9,9,79.0
7,2019,2836,8,8,80.0
8,2019,2836,9,9,80.0
9,2019,2843,10,10,55.0


The table above shows the number of months for which Digital Platform data and House prices data were jointly available. Due to data limitation, unfortunately, the two information sources were jointly available for a limited amount of time.

In [28]:
df_merged_rome[["ASSET_PRICE", "EURO_MQ"]].corr()

,ASSET_PRICE,EURO_MQ
ASSET_PRICE,1.000000,0.419751
EURO_MQ,0.419751,1.000000


Despite the data limitation we attempted at computing a correlation across the two variables. The table above shows a positive correlation across asset price and house prices for the Rome case. 

These results have to be interpreted only as a data experiment. Additional information, and longer time series, are indeed required to further explore this preliminary result

In [29]:
df_merged_madrid = df_madrid[[c for c in list(df_madrid) if c !='MONTH']].merge(df_price_bnb_madrid, left_on = ['YEAR', 'N_MONTH'], 
                          right_on = ['YEAR', 'MONTH'], how='inner')

In [30]:
df_merged_madrid.shape

(23, 5)

In [31]:
df_merged_madrid.head()

,YEAR,EURO_MQ,N_MONTH,MONTH,ASSET_PRICE
0,2018,3125.0,1,1,63.0
1,2018,3310.0,4,4,65.0
2,2018,3406.0,5,5,65.0
3,2018,3550.0,7,7,65.0
4,2018,3572.0,8,8,65.0


The figure above shows that Madrid has even further data limitations when it comes to measuring the correlation of house prices and asset prices.

In [32]:
df_merged_madrid[["ASSET_PRICE", "EURO_MQ"]].corr()

,ASSET_PRICE,EURO_MQ
ASSET_PRICE,1.000000,0.575337
EURO_MQ,0.575337,1.000000


The table above shows the correlation across the two variables for the Madrid case. These results have to be considered bearing in mind of the data limitations described above.

# Conclusions

The data experiment reported in this notebook shows that it is indeed possible, provided that the right data is available, to measure a potential correlation across asset price and house prices. 